<a href="https://colab.research.google.com/github/emirunlu26/future-context-ST/blob/main/code/asr_mt_test_on_whisper_nllb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:

threshold = 750 # number of segment informations to read

In [ ]:
import numpy as np
import librosa

duration_class_by_upper_bound = {"1":5,"2":10,"3":15,"4":20}
duration_class_by_size = {"1":0,"2":0,"3":0,"4":0}
duration_class_by_percentage = {"1":0,"2":0,"3":0,"4":0}

highest_duration = 0

for index in range(threshold):
  audio_name = "audio_" + str(index) + ".wav"
  audio_path = "/content/drive/MyDrive/ColabNotebooks/audio_segments_test_set/" + audio_name
  audio, sample_rate = librosa.load(audio_path,sr=None)
  duration = len(audio) / sample_rate

  if duration > highest_duration:
    highest_duration = duration

  for duration_class in duration_class_by_upper_bound:
    if duration < duration_class_by_upper_bound[duration_class]:
      duration_class_by_size[duration_class] += 1
      break
print("\n\n")
print("Highest duration is:", highest_duration)
print(duration_class_by_size)

for duration_class in duration_class_by_percentage:
  duration_class_by_percentage[duration_class] = round(duration_class_by_size[duration_class] / threshold,3) * 100

print(duration_class_by_percentage)









Highest duration is: 19.63995464852608
{'1': 203, '2': 303, '3': 187, '4': 57}
{'1': 27.1, '2': 40.400000000000006, '3': 24.9, '4': 7.6}


In [ ]:
import numpy as np
import librosa
import soundfile as sf

start_index = 500
threshold = 750

# STORE THE SEGMENT INFORMATIONS IN segment_infos LIST
segment_infos = list()

with open("/content/drive/MyDrive/ColabNotebooks/segments.lst","r") as segment_file:
  segment_lines = segment_file.readlines()

  print(f"Total number of segment infos is {len(segment_lines)}")
  segment_lines = segment_lines[start_index:]
  for line in segment_lines:
    segment_info = line[0:len(line)-1].split()
    segment_infos.append(segment_info)

    if len(segment_infos) == threshold:
      break

  segment_infos = np.array(segment_infos)

  print(f"Length of segment_infos is {len(segment_infos)}")

  '''
  last_unique_audio = None
  unique_audioset = list()
  for segment_info in segment_infos:
    if last_unique_audio == segment_info[0]:
      continue
    unique_audioset.append(segment_info[0])
    last_unique_audio = segment_info[0]
    print(segment_info[0])
  '''

# GET THE AUDIO DATA FOR THE AUDIO SEGMENTS
  audio_segments = list()

  for index, segment in enumerate(segment_infos):
    audio_path = "/content/drive/MyDrive/ColabNotebooks/audios_test_set/" + segment[0] + ".m4a"
    audio_data, sample_rate = librosa.load(audio_path)
    start_sec = float(segment[1])
    end_sec = float(segment[2])
    start_sample = int(start_sec * sample_rate)
    end_sample = int(end_sec * sample_rate)
    audio_segment = audio_data[start_sample:end_sample]

    # Define output path
    output_path = f"/content/drive/MyDrive/ColabNotebooks/audio_segments_test_set/audio_{index}.wav"

    # Save the segment
    sf.write(output_path, audio_segment, sample_rate)
    print(f"Audio number {index+1} written ...")

    audio_segments.append(audio_segment)

Total number of segment infos is 1253
Length of segment_infos is 750


<ipython-input-8-09ba9807059e>:43: UserWarning: PySoundFile failed. Trying audioread instead.
  audio_data, sample_rate = librosa.load(audio_path)


Audio number 1 written ...


<ipython-input-8-09ba9807059e>:43: UserWarning: PySoundFile failed. Trying audioread instead.
  audio_data, sample_rate = librosa.load(audio_path)
/usr/local/lib/python3.11/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


Audio number 2 written ...


...

Audio number 748 written ...


<ipython-input-8-09ba9807059e>:43: UserWarning: PySoundFile failed. Trying audioread instead.
  audio_data, sample_rate = librosa.load(audio_path)
/usr/local/lib/python3.11/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


Audio number 749 written ...


<ipython-input-8-09ba9807059e>:43: UserWarning: PySoundFile failed. Trying audioread instead.
  audio_data, sample_rate = librosa.load(audio_path)
/usr/local/lib/python3.11/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


Audio number 750 written ...


**en.20101216.19.4-271.m4a**

GET THE TRANSCRIPTIONS BELONGING TO THE FIRST 500 AUDIO SEGMENTS

In [ ]:
with open("/content/drive/MyDrive/ColabNotebooks/segments.en","r") as transcr_file:
  ref_transcriptions = list()
  for transcription in transcr_file:
    transcription = transcription[:len(transcription)-1]
    ref_transcriptions.append(transcription)
    if len(ref_transcriptions) == threshold:
      break

GET THE GERMAN TRANSLATIONS OF THE FIRST 500 TRANSCRIPTIONS

In [ ]:
with open("/content/drive/MyDrive/ColabNotebooks/segments.de","r") as translate_file:
  ref_translations = list()
  for translation in translate_file:
    translation = translation[:len(translation)-1]
    ref_translations.append(translation)
    if len(ref_translations) == threshold:
      break
  ref_translations = [ref_translations]

**GET MODEL BY USING PIPELINE**

In [ ]:
!pip install transformers
!pip install -U openai-whisper
!pip install --upgrade soundfile

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 49.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 67.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 123.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 92.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 58.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 40.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.2 M

In [ ]:
import torch
from transformers import pipeline

whisper = pipeline("automatic-speech-recognition", "openai/whisper-large-v3", torch_dtype=torch.float16, device="cuda:0")

whisper_transcriptions = map(whisper, audio_segments)
whisper_transcriptions = list(map(lambda x: x["text"], whisper_transcriptions))

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.90k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

...

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.07k [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/340 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/whisper/generation_whisper.py:509: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.43.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


**GET MODEL BY CALLING whisper.load_model()**

In [ ]:
import whisper
import librosa
import numpy as np

model = whisper.load_model("turbo").to("cuda")
whisper_transcriptions = list()

for index in range(threshold):
  audio_path = "/content/drive/MyDrive/ColabNotebooks/audio_segments_new/audio_" + str(index) + ".wav"
  audio, sample_rate = librosa.load(audio_path,sr=None)
  audio = whisper.pad_or_trim(np.array(audio).astype("float32"))
  mel = whisper.log_mel_spectrogram(audio,n_mels=model.dims.n_mels).to(model.device)

  options = whisper.DecodingOptions(language="en")
  result = whisper.decode(model, mel, options)
  whisper_transcriptions.append(result.text)

100%|█████████████████████████████████████| 1.51G/1.51G [00:36<00:00, 44.3MiB/s]
/usr/local/lib/python3.11/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this exper

**LOAD THE PREDICTIONS GENERATED BY WHISPER IN SIMULEVAL**

In [ ]:
with open("/content/drive/MyDrive/ColabNotebooks/SimulEval/Whisper-Wait-k/predicted_transcriptions.txt","r") as whisper_simuleval_predictions:
  simuleval_transcriptions = whisper_simuleval_predictions.readlines()
  simuleval_transcriptions = list(map(lambda x: x[:len(x)-1], simuleval_transcriptions))

**EVALUATE SIMULEVAL WHISPER RESULTS**

In [ ]:
import numpy as np
from jiwer import wer, mer, wil, wip, cer

# CALCULATE WORD ERROR RATE
wer_scores = list()

for ref, pred in zip(ref_transcriptions, simuleval_transcriptions):
  wer_scores.append(wer(ref, pred))

wer_scores = np.array(wer_scores)
print(wer_scores.mean())


0.4982513039820344


EVALUATE WHISPER BY USING **jiwer**

In [ ]:
!pip install jiwer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 64.6 MB/s eta 0:00:00


In [ ]:
import numpy as np
from jiwer import wer, mer, wil, wip, cer

# CALCULATE WORD ERROR RATE
wer_scores = list()

for ref, pred in zip(ref_transcriptions, whisper_transcriptions):
  wer_scores.append(wer(ref, pred))

wer_scores = np.array(wer_scores)

# CALCULATE MATCH ERROR RATE
mer_scores = list()

for ref, pred in zip(ref_transcriptions, whisper_transcriptions):
  mer_scores.append(mer(ref, pred))

mer_scores = np.array(mer_scores)

# CALCULATE WORD INFORMATION LOST
wil_scores = list()

for ref, pred in zip(ref_transcriptions, whisper_transcriptions):
  wil_scores.append(wil(ref, pred))

wil_scores = np.array(wil_scores)

# CALCULATE WORD INFORMATION PRESERVED

wip_scores = np.ones(len(wil_scores)) - wil_scores

# CALCULATE CHARACTER ERROR RATE
cer_scores = list()

for ref, pred in zip(ref_transcriptions, whisper_transcriptions):
  cer_scores.append(cer(ref, pred))

cer_scores = np.array(cer_scores)

In [ ]:
print(wer_scores.mean())
print(wil_scores.mean())
print(wip_scores.mean())

0.1935002612561314
0.2697960478532376
0.7302039521467624


STORE THE MEAN OF EACH EVALUATION METRIC IN A DATAFRAME

In [ ]:
import pandas as pd

whisper_eval_results = {"WER": wer_scores.mean(), "MER": mer_scores.mean(), "WIL": wil_scores.mean(), "WIP": wip_scores.mean(), "CER": cer_scores.mean()}
#whisper_eval_results = {"WER": wer_scores.mean()}

asr_eval_results_df = pd.DataFrame(whisper_eval_results, index=["Whisper Turbo"])
asr_eval_results_df.index.name = "ASR Model"
asr_eval_results_df.title = "ASR Evaluation Results on First 30 Audio Segments"

asr_eval_results_df.to_csv("/content/drive/MyDrive/ColabNotebooks/asr_eval_results.csv")

GET THE TRANSLATION PREDICTIONS GENERATED BY **NLLB** (INPUT: TRANSCRIPTIONS PROVIDED BY EUROPARL)

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

model_name = "facebook/nllb-200-distilled-600M"
tokenizer = AutoTokenizer.from_pretrained(model_name,src_lang="eng_Latn")
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to("cuda")

# DEFINE FUNCTION THAT APPLIES TRANSLATION

def from_eng_to_ger(eng_text):
  input_tokens = tokenizer(eng_text, return_tensors="pt").to("cuda")
  output_tokens = model.generate(
    **input_tokens, forced_bos_token_id=tokenizer.convert_tokens_to_ids("deu_Latn"), max_length=50)
  ger_text = tokenizer.batch_decode(output_tokens, skip_special_tokens=True)[0]
  return ger_text

# TRANSLATE SEGMENTED TEXTUAL DATA BY USING NLLB MODEL

source_texts = ref_transcriptions
nllb_translations = list(map(from_eng_to_ger, source_texts))

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/564 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/4.85M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.3M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/3.55k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/846 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

model_name = "facebook/nllb-200-distilled-600M"
tokenizer = AutoTokenizer.from_pretrained(model_name,src_lang="eng_Latn")
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to("cuda")

text = ref_transcriptions[22]
source_text = text
source_text_encoding = tokenizer(source_text,return_tensors = "pt").to("cuda")

output_tokens = model.generate(
            **source_text_encoding,
            decoder_input_ids = None,
            forced_bos_token_id = tokenizer.convert_tokens_to_ids("deu_Latn"),
            max_new_tokens = 50,
        )

output_text = tokenizer.batch_decode(output_tokens, skip_special_tokens=True)[0]

print("Input id:", source_text_encoding["input_ids"])
print()
print("Output text:", output_text)
print()
print("Output tokens:", output_tokens)

Input id: tensor([[256047,   2188,  33726,   1830,   4062,    280,  30598,   8362, 248079,
           3605,    796,    248,  60372, 248079,    540,    108,  15807,  19450,
          38011,   6370,    248,  28911,  54764,   1482,    248,  12625,   6780,
          30214,    138,   3605,   4062,   2294,    280,  57758,    108,  15807,
         103441, 248075, 248059,      2]], device='cuda:0')

Output text: Die Anpassung wird schmerzhaft sein, aber sie ist notwendig, und in diesen schwierigen Zeiten gibt es eine weitere Botschaft, die noch deutlicher ist, aber in diesen Institutionen nicht gehört wird. 

Output tokens: tensor([[     2, 256042,   2667,    544,   2121,  12137,  11590,   4055,   1560,
         101667,   1210,  11160, 248079,  15704,   2118,   1876,   2294, 125109,
         248079,   1839,    108,  50799,   4055,  35763,  11291,   3636,   5039,
          38145,    335,   7458, 151740, 104285,  13972, 248079,    566,  12501,
         145225,  59068,   1876, 248079,  15704,    

**GET THE TRANSLATION PREDICTIONS GENERATED BY MARIAN NMT (INPUT: TRANSCRIPTIONS PROVIDED BY EUROPARL)**

In [ ]:
from transformers import MarianMTModel, MarianTokenizer

model_name = "Helsinki-NLP/opus-mt-en-de"
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name).to("cuda")

def from_eng_to_ger_marian(eng_text):
  input_tokens = tokenizer(eng_text, return_tensors="pt", padding=True).to("cuda")
  output_tokens = model.generate(**input_tokens)
  ger_text = tokenizer.batch_decode(output_tokens, skip_special_tokens=True)[0]
  return ger_text

source_texts = ref_transcriptions
marian_translations = list(map(from_eng_to_ger_marian, source_texts))

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/768k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/797k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.27M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.33k [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


pytorch_model.bin:   0%|          | 0.00/298M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

**EVALUATION OF SIMULEVAL TRANSLATIONS**

In [ ]:
path = "/content/drive/MyDrive/ColabNotebooks/SimulEval/NLLB-Wait-k/nllb_output.txt"
#path = "/content/drive/MyDrive/ColabNotebooks/results/nllb_online_ref_transcriptions (simuleval)/nllb_k_3_output_2/output.txt"
with open(path,"r") as simuleval_file:
  simuleval_translations = simuleval_file.readlines()
  simuleval_translations = list(map(lambda x: x[:len(x)-1], simuleval_translations))

In [ ]:
with open("/content/drive/MyDrive/ColabNotebooks/SimulEval/NLLB-Wait-k/nllb_k_5_output_2.txt","r") as simuleval_file:
  simuleval_translations_2 = simuleval_file.readlines()
  simuleval_translations_2 = list(map(lambda x: x[:len(x)-1], simuleval_translations_2))

In [ ]:
import numpy as np
from jiwer import wer, mer, wil, wip, cer

# CALCULATE WORD ERROR RATE
wer_scores = list()
upper_limit = 0.9
count = 0

for i in range(500):
  wer_score = wer(simuleval_translations[i], simuleval_translations_2[i])
  if wer_score > upper_limit:
    count +=1
    print("Index:", i)
    print("WER:", wer_score)
    print("Translation 1:", simuleval_translations[i])
    print("Translation 2:", simuleval_translations_2[i] + "\n")
print("\nCount:", count)

ModuleNotFoundError: No module named 'jiwer'

In [ ]:
import sacrebleu

nllb_bleu = sacrebleu.corpus_bleu(simuleval_translations, ref_translations)

print(nllb_bleu.score)
print(nllb_bleu)

19.433925648197434
BLEU = 19.43 55.2/27.1/15.4/9.3 (BP = 0.904 ratio = 0.908 hyp_len = 11593 ref_len = 12765)


**EVALUATION OF SIMULEVAL TRANSLATIONS - THE END**

EVALUATE NLLB BY USING **sacreBLEU** (INPUT: TRANSCRIPTIONS PROVIDED BY EUROPARL)

In [ ]:
!pip install sacrebleu

In [ ]:
import sacrebleu

nllb_bleu = sacrebleu.corpus_bleu(nllb_translations, ref_translations)
print(nllb_bleu)

BLEU = 22.98 56.3/31.4/19.9/12.9 (BP = 0.886 ratio = 0.892 hyp_len = 11389 ref_len = 12765)


In [ ]:
import sacrebleu
import pandas as pd

nllb_bleu = sacrebleu.corpus_bleu(nllb_translations, ref_translations)

nllb_bleu_df = pd.DataFrame({"sacreBLEU(on 500 segments)": nllb_bleu.score}, index=["nllb-200-distilled-600M"])
nllb_bleu_df.index.name = "MT Model"

nllb_bleu_df.to_csv("/content/drive/MyDrive/ColabNotebooks/nllb_bleu.csv")


In [ ]:
with open("/content/drive/MyDrive/ColabNotebooks/nllb_offline_output.txt", "w", encoding="utf-8") as target:
  for translation in nllb_translations:
    target.write(translation + "\n")

**GET THE TRANSLATION PREDICTIONS GENERATED BY NLLB (INPUT: TRANSCRIPTIONS GENERATED BY WHISPER)**

In [ ]:
nllb_translations = list(map(from_eng_to_ger, whisper_transcriptions))

In [ ]:
with open("/content/drive/MyDrive/ColabNotebooks/results/nllb_offline_hypo_transcriptions/output.txt", "w", encoding="utf-8") as target:
  for translation in nllb_translations:
    target.write(translation + "\n")

**EVALUATE NLLB BY USING sacreBLEU (INPUT: TRANSCRIPTIONS GENERATED BY WHISPER)**

In [ ]:
import sacrebleu
import pandas as pd

nllb_bleu = sacrebleu.corpus_bleu(nllb_translations, ref_translations)

nllb_bleu_df = pd.DataFrame({"sacreBLEU(on 500 segments)": nllb_bleu.score}, index=["nllb-200-distilled-600M"])
nllb_bleu_df.index.name = "MT Model"

nllb_bleu_df.to_csv("/content/drive/MyDrive/ColabNotebooks/evaluations/nllb_offline_hypo_transcriptions/nllb_bleu_hypothesis_input.csv")


**COMPARE OUTPUTS OF DIFFERENT CASCADED SETUPS (MODELS NOT INTERCONNECTED)**

In [ ]:
with open("/content/drive/MyDrive/ColabNotebooks/SimulEval/NLLB-Wait-k/translation_simuleval_2.txt","r") as simuleval_file:
  simuleval_translations = simuleval_file.readlines()
  simuleval_translations = list(map(lambda x: x[:len(x)-1], simuleval_translations))

In [ ]:
with open("/content/drive/MyDrive/ColabNotebooks/SimulEval/NLLB-Wait-k/translation_simuleval_4.txt","r") as simuleval_file:
  simuleval_translations_2 = simuleval_file.readlines()
  simuleval_translations_2 = list(map(lambda x: x[:len(x)-1], simuleval_translations_2))

In [ ]:
import numpy as np
from jiwer import wer, mer, wil, wip, cer

# CALCULATE WORD ERROR RATE
wer_scores = list()

for ref, pred in zip(simuleval_translations, simuleval_translations_2):
  wer_scores.append(wer(ref, pred))

wer_scores = np.array(wer_scores)
print(wer_scores.mean())

0.2572314497124328


In [ ]:
import sacrebleu

nllb_bleu = sacrebleu.corpus_bleu(simuleval_translations, ref_translations)

print(nllb_bleu.score)

72.59795291154772


In [ ]:
import sacrebleu

nllb_bleu = sacrebleu.corpus_bleu(simuleval_translations_2, ref_translations)

print(nllb_bleu.score)

72.59795291154772


**CALCULATE BLEU SCORE OF OUTPUTS FROM REAL CASCADED SETUPS (MODELS INTERCONNECTED)**

In [ ]:
with open("/content/drive/MyDrive/ColabNotebooks/SimulEval/Cascaded-Wait-k/cascaded_setup_2_results.txt","r") as cascaded_output_file:
  cascaded_translations = cascaded_output_file.readlines()
  cascaded_translations = list(map(lambda x: x[:len(x)-1], cascaded_translations))

In [ ]:
import sacrebleu

nllb_bleu = sacrebleu.corpus_bleu(cascaded_translations, ref_translations)

print(nllb_bleu.score)

49.719876934333


**COMPARE OUTPUTS FROM REAL CASCADED SETUPS WITH EACH OTHER (MODELS INTERCONNECTED)**

In [ ]:
with open("/content/drive/MyDrive/ColabNotebooks/SimulEval/Cascaded-Wait-k/cascaded_setup_3_results.txt","r") as cascaded_output_file:
  cascaded_translations_1 = cascaded_output_file.readlines()
  cascaded_translations_1 = list(map(lambda x: x[:len(x)-1], cascaded_translations_1))

In [ ]:
with open("/content/drive/MyDrive/ColabNotebooks/SimulEval/Cascaded-Wait-k/cascaded_setup_4_results.txt","r") as cascaded_output_file_2:
  cascaded_translations_2 = cascaded_output_file_2.readlines()
  cascaded_translations_2 = list(map(lambda x: x[:len(x)-1], cascaded_translations_2))

In [ ]:
import numpy as np
from jiwer import wer, mer, wil, wip, cer

# CALCULATE WORD ERROR RATE
wer_scores = list()

for ref, pred in zip(cascaded_translations_1, cascaded_translations_2):
  wer_scores.append(wer(ref, pred))

wer_scores = np.array(wer_scores)
print(wer_scores.mean())

0.25433563781408325
